In [120]:
import os
import numpy as np

base_path = '/mnt/disk1/heonseok/MPMLD/output'

In [121]:
dataset = 'location'

z_dim_list = [
    # '16',
    '64',
]

setsize_list = [
    # '500',
    # '1000',
    '2000',
]

lr_list = [
    '0.001',
    # '0.01',
    # '0.1',
]

ref_list = [
    '0.1',
]

arc_list = [
    # 'A',
    # 'B',
    'C',
]

dis_type_list = [
    # 'base',
    'type5',
]

cw_list = [
    '0.01',
    '0.1',
    '1.0',
]

mw_list = [
    '0.01',
    '0.1',
    '1.0',
]

recon_model_list = []
for z_dim in z_dim_list:
    for setsize in setsize_list:
        for lr in lr_list:
            for ref in ref_list:
                for arc in arc_list:
                    for dis_type in dis_type_list:
                        if 'base' in dis_type:
                            recon_model = 'AE_z{}_setsize{}_lr{}_ref{}_arc{}_{}'.format(
                                z_dim, setsize, lr, ref, arc, dis_type,
                            )
                            recon_model_list.append(recon_model)
                            continue
                        for cw in cw_list:
                            for mw in mw_list:
                                recon_model = 'AE_z{}_setsize{}_lr{}_ref{}_arc{}_{}_cw{}_mw{}'.format(
                                    z_dim, setsize, lr, ref, arc, dis_type, cw, mw,
                                )
                                recon_model_list.append(recon_model)

for recon_model in recon_model_list:
    print('\''+recon_model+'\',')

class_model_list = []
for idx in range(len(recon_model_list)):
    class_model_list.append(recon_model_list[idx] + '_FCNClassifierA')
    print('\''+class_model_list[idx]+'\',')


'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw0.01_mw0.01',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw0.01_mw0.1',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw0.01_mw1.0',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw0.1_mw0.01',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw0.1_mw0.1',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw0.1_mw1.0',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw1.0_mw0.01',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw1.0_mw0.1',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw1.0_mw1.0',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw0.01_mw0.01_FCNClassifierA',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw0.01_mw0.1_FCNClassifierA',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw0.01_mw1.0_FCNClassifierA',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw0.1_mw0.01_FCNClassifierA',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw0.1_mw0.1_FCNClassifierA',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw0.1_mw1.0_FCNClassifierA',

In [122]:
for recon_model in recon_model_list:
    print('\''+recon_model+'\',')

for recon_model in recon_model_list:
    recon_path = os.path.join(base_path, dataset, 'reconstructor', recon_model)

    class_acc_full_list = []
    class_acc_content_list = []
    class_acc_style_list = []
    membership_acc_full_list = []
    membership_acc_content_list = []
    membership_acc_style_list = []

    for repeat in range(5):
        try:
            recon_repeat_path = os.path.join(recon_path, 'repeat{}'.format(repeat))
            recon_acc_path = os.path.join(recon_repeat_path, 'acc.npy')
            recon_acc = np.load(recon_acc_path, allow_pickle=True).item()

            class_acc_full_list.append(recon_acc['class_acc_full'])
            class_acc_content_list.append(recon_acc['class_acc_content'])
            class_acc_style_list.append(recon_acc['class_acc_style'])
            membership_acc_full_list.append(recon_acc['membership_acc_full'])
            membership_acc_content_list.append(recon_acc['membership_acc_content'])
            membership_acc_style_list.append(recon_acc['membership_acc_style'])

        except FileNotFoundError:
            # print('File not found: ', recon_acc_path)
            continue

    result_list = [
        *recon_model.split('_'),
        np.average(class_acc_full_list), np.average(class_acc_content_list), np.average(class_acc_style_list),
        np.average(membership_acc_full_list), np.average(membership_acc_content_list), np.average(membership_acc_style_list)
    ]

    for idx, result in enumerate(result_list):
        if not type(result) is str:
            result_list[idx] = '{:.4f}'.format(result)
    print(','.join(result_list))


AE,z64,setsize2000,lr0.001,ref0.1,arcC,type5,cw0.01,mw0.01,0.8648,0.7384,0.1299,0.9468,0.8252,0.8891
AE,z64,setsize2000,lr0.001,ref0.1,arcC,type5,cw0.01,mw0.1,0.8575,0.7211,0.1383,0.8347,0.5001,0.8200
AE,z64,setsize2000,lr0.001,ref0.1,arcC,type5,cw0.01,mw1.0,0.8519,0.7232,0.1353,0.6627,0.5000,0.8039
AE,z64,setsize2000,lr0.001,ref0.1,arcC,type5,cw0.1,mw0.01,0.8434,0.7628,0.0671,0.9149,0.8406,0.8485
AE,z64,setsize2000,lr0.001,ref0.1,arcC,type5,cw0.1,mw0.1,0.7864,0.7111,0.0647,0.6251,0.5161,0.7657
AE,z64,setsize2000,lr0.001,ref0.1,arcC,type5,cw0.1,mw1.0,0.8397,0.7250,0.0676,0.5000,0.5000,0.7719
AE,z64,setsize2000,lr0.001,ref0.1,arcC,type5,cw1.0,mw0.01,0.9057,0.7213,0.0655,0.9254,0.8527,0.6749
AE,z64,setsize2000,lr0.001,ref0.1,arcC,type5,cw1.0,mw0.1,0.8589,0.6660,0.0655,0.7097,0.5000,0.6473
AE,z64,setsize2000,lr0.001,ref0.1,arcC,type5,cw1.0,mw1.0,0.8531,0.7120,0.0655,0.4999,0.5000,0.6651


In [131]:
for class_model in class_model_list:
    print('\''+class_model+'\',')

for class_model in class_model_list:
    class_path = os.path.join(base_path, dataset, 'classifier', class_model)

    result_list = []
    for recon_type in ['full_z', 'content_z', 'style_z']:
        train_acc_list = []
        valid_acc_list = []
        test_acc_list = []
        for repeat in range(5):
            try:
                class_repeat_path = os.path.join(class_path, recon_type, 'repeat{}'.format(repeat))
                class_acc = np.load(os.path.join(class_repeat_path, 'acc.npy'), allow_pickle=True).item()
                train_acc_list.append(class_acc['train'])
                valid_acc_list.append(class_acc['valid'])
                test_acc_list.append(class_acc['test'])

            except FileNotFoundError:
                # print('File not found: ', class_repeat_path)
                continue

        result_list.extend([np.average(train_acc_list), np.average(valid_acc_list), np.average(test_acc_list)])

    for idx, result in enumerate(result_list):
        if not type(result) is str:
            result_list[idx] = '{:.4f}'.format(result)
    print(','.join(result_list))


'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw0.01_mw0.01_FCNClassifierA',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw0.01_mw0.1_FCNClassifierA',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw0.01_mw1.0_FCNClassifierA',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw0.1_mw0.01_FCNClassifierA',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw0.1_mw0.1_FCNClassifierA',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw0.1_mw1.0_FCNClassifierA',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw1.0_mw0.01_FCNClassifierA',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw1.0_mw0.1_FCNClassifierA',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw1.0_mw1.0_FCNClassifierA',
0.8139,0.6665,0.6220,0.5867,0.4000,0.3265,0.0674,0.0775,0.0655
0.8270,0.6670,0.6160,0.5078,0.2760,0.2235,0.0669,0.0735,0.0575
0.7867,0.6770,0.6235,0.6004,0.3685,0.3195,0.0693,0.0780,0.0555
0.8319,0.6700,0.6170,0.6317,0.3545,0.3095,0.0726,0.0795,0.0565
0.8004,0.6535,0.6250,0.4343,0.2460,0.2245,0.0637,0.0765,0.0630
0.79

In [133]:
attack_type_list = ['stat', 'black']
for class_model in class_model_list:
    print('\''+class_model+'\',')

for class_model in class_model_list:
    attack_path = os.path.join(base_path, dataset, 'attacker', class_model)

    result_list = []
    for recon_type in ['full_z', 'content_z', 'style_z']:
        stat_acc_list = []
        black_acc_list = []
        # white_acc_list = []
        for repeat in range(5):
            for attack_type in attack_type_list:
                attack_repeat_path = os.path.join(attack_path, recon_type, 'repeat{}'.format(repeat))
                try:
                    attack_acc = np.load(os.path.join(attack_repeat_path, attack_type, 'acc.npy'), allow_pickle=True)
                    if attack_type == 'stat':
                        stat_acc_list.append(attack_acc)
                    elif attack_type == 'black':
                        black_acc_list.append(attack_acc.item()['test'])
                    # elif attack_type == 'white':
                    #     white_acc_list.append(attack_acc.item()['test'])

                except FileNotFoundError:
                    print('File not found: ', attack_repeat_path)
                    continue

        result_list.extend([np.average(stat_acc_list), np.average(black_acc_list)])

    for idx, result in enumerate(result_list):
        if not type(result) is str:
            result_list[idx] = '{:.4f}'.format(result)
    print(','.join(result_list))


'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw0.01_mw0.01_FCNClassifierA',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw0.01_mw0.1_FCNClassifierA',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw0.01_mw1.0_FCNClassifierA',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw0.1_mw0.01_FCNClassifierA',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw0.1_mw0.1_FCNClassifierA',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw0.1_mw1.0_FCNClassifierA',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw1.0_mw0.01_FCNClassifierA',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw1.0_mw0.1_FCNClassifierA',
'AE_z64_setsize2000_lr0.001_ref0.1_arcC_type5_cw1.0_mw1.0_FCNClassifierA',
0.5529,0.5767,0.5183,0.5100,0.5139,0.5143
0.5546,0.5803,0.5130,0.5197,0.5119,0.5330
0.5504,0.5617,0.5134,0.5230,0.5180,0.5037
0.5584,0.5733,0.5143,0.5163,0.5156,0.5110
0.5514,0.5617,0.5102,0.5170,0.5162,0.5130
0.5486,0.5697,0.5092,0.5133,0.5125,0.5193
0.5584,0.5733,0.5185,0.5173,0.5291,0.5163
0.5567,0.5777,0.5131,0.51

In [140]:
classifier_with_raw_data = 'original_setsize2000_FCNClassifierA'
class_path = os.path.join(base_path, dataset, 'classifier', classifier_with_raw_data)
attack_path = os.path.join(base_path, dataset, 'attacker', classifier_with_raw_data)
train_acc_list = []
valid_acc_list = []
test_acc_list = []
stat_acc_list = []
black_acc_list = []
for repeat in range(5):
    try:
        class_repeat_path = os.path.join(class_path, 'repeat{}'.format(repeat))
        attack_repeat_path = os.path.join(attack_path, 'repeat{}'.format(repeat))

        class_acc = np.load(os.path.join(class_repeat_path, 'acc.npy'), allow_pickle=True).item()
        train_acc_list.append(class_acc['train'])
        valid_acc_list.append(class_acc['valid'])
        test_acc_list.append(class_acc['test'])

        for attack_type in attack_type_list:
            attack_acc = np.load(os.path.join(attack_repeat_path, attack_type, 'acc.npy'), allow_pickle=True)
            if attack_type == 'stat':
                stat_acc_list.append(attack_acc)
            elif attack_type == 'black':
                black_acc_list.append(attack_acc.item()['test'])

    except FileNotFoundError:
        print('File not found: ', class_repeat_path)
        continue

print('{:.4f} {:.4f} {:.4f} {:.4f} {:.4f}'.format(
    np.average(train_acc_list), np.average(valid_acc_list), np.average(test_acc_list),
    np.average(stat_acc_list), np.average(black_acc_list),
))
# print(np.average(train_acc_list), np.average(valid_acc_list), np.average(test_acc_list))
# print(np.average(stat_acc_list), np.average(black_acc_list), np.average(test_acc_list))



0.9420 0.6850 0.6535 0.6066 0.6743
